In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#making necessary imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os.path

In [ ]:
holidays=pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil=pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores=pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
test=pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
train=pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
transactions=pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
test1=pd.read_csv('../input/store-sales-time-series-forecasting/test.csv') #creating another copy of test data which will be used later.

In [ ]:
#Merging all available datasets to perform exploratory data analysis
m1=pd.merge(holidays,oil)
m2=pd.merge(train,stores)
m3=pd.merge(m2,transactions)
df=pd.merge(m3,m1,on="date")

In [ ]:
df.head()

# **Exploratory Data Analysis**

In [ ]:
df.describe()

We observe null values are present in the "daily oil prices column" or "dcoilwtico".

In [ ]:
#Replacing the null values with the mean daily oil prices
df.loc[(df.dcoilwtico.isnull()),'dcoilwtico']=df.dcoilwtico.mean()

In [ ]:
#checking the dataset
df.info()

There are no null values in the dataset.

In [ ]:
#Converting the date column from string to datetime dtype.
from datetime import datetime
df['new_date']=pd.to_datetime(df['date'],format='%Y-%m-%d',errors='coerce')

In [ ]:
#Time Series plot of the sales data
sns.lineplot(x='new_date',y='sales',data=df,ci=None,estimator='mean')

The plot shows that there is an increasing trend of sales with each year. Irregularities and fluctuations are also observed. From 2013 to 2017, the sales have increased, thus showing an upward trend.

**Next, we will try to analyse the monthwise sales to find out which months on an average have contributed to high sales. This can be done by splitting the 'date' column into year, month and day columns.**

In [ ]:
df[["year", "month", "day"]] = df["date"].str.split("-", expand = True)


df['month'].replace(['01','02','03','04','05','06','07','08','09','10','11','12'],['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec'],inplace=True)
df['month'] = pd.Categorical(df['month'],
                                   categories=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec'],
                                   ordered=True)
df['day'] = pd.Categorical(df['day'],categories=['01','02','03','04','05','06','07','08','09','10','11', '12', '14','15','16','17','18','19','20','21','22','23', '24', '25', '26', '27','28','29','30','31'],ordered=True)
        

In [ ]:
#plotting the monthwise sales trend
f,ax=plt.subplots(1,2,figsize=(25,15))
sns.lineplot(x='year',y='sales',data=df,ci=None,estimator='mean',ax=ax[0])
sns.lineplot(x='month',y='sales',data=df,ci=None,estimator='mean',ax=ax[1])

As before, an upward trend is observed yearwise. For months, on an average, there has been the highest sales in the month of January with sharp dips in February and June (minimum average sales) and again increased in December.

**Let's analyse the effects of 'family'and 'store_nbr' on sales.**

In [ ]:
df['family'].nunique()

There are 33 unique categories in family which can be re-classified into groups as follows.

In [ ]:
df['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)
df['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)
df['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)
df['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)
df['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

In [ ]:
#Preparing the train and test datasets which would be used for prediction later

train['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)
train['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)
train['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)
train['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)
train['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

test['family'].replace(['AUTOMOTIVE', 'HARDWARE', 'LAWN AND GARDEN', 'PLAYERS AND ELECTRONICS'], 'Tools', inplace = True)
test['family'].replace(['BEAUTY', 'LINGERIE', 'LADIESWEAR','PERSONAL CARE','CELEBRATION','MAGAZINES','BOOKS', 'BABY CARE'], 'LifeStyle', inplace = True)
test['family'].replace(['HOME APPLIANCES','HOME AND KITCHEN I', 'HOME AND KITCHEN II','HOME CARE','SCHOOL AND OFFICE SUPPLIES'], 'Home', inplace=True)
test['family'].replace([ 'GROCERY II', 'PET SUPPLIES','SEAFOOD','LIQUOR,WINE,BEER'], 'Food', inplace=True)
test['family'].replace(['DELI', 'EGGS'], 'Daily', inplace=True)

In [ ]:
#Plotting the sales using bar-plot w.r.t to 'family' categories.

d=df.groupby('family')['sales'].mean().sort_values(ascending=False)
d=pd.DataFrame(d)
df.sales= pd.to_numeric(df.sales, errors='coerce')
ax,f=plt.subplots(figsize=(25,15))
sns.barplot(x='sales',y=d.index,data=d,ax=None,ci=None)

Maximum sales are there for 'Grocery I' followed by 'Beverages', ;Produce', 'Cleaning' , etc. The minimum is in case of 'Tools'.

In [ ]:
#Plot of sales w.r.t store number

s=df.groupby('store_nbr')['sales'].mean().sort_values(ascending=False)
s=pd.DataFrame(s)
ax,f=plt.subplots(figsize=(25,15))
sns.barplot(x=s.index,y='sales',data=s,ax=None,ci=None,order=s.index)

Highest sales are for store numbers 44,45,47,etc. and lowest are for 30,32,etc.

Let's see which city and states correspond to the highest and lowest sales store numbers.

In [ ]:
#For that,we will make new columns

df['city_state']=df['city']+"_"+df['state']
df['type_locale']=df['type_y']+"_"+"locale"

city_state=df[(df['store_nbr']==44)|(df['store_nbr']==45)|(df['store_nbr']==47)|(df['store_nbr']==30)|(df['store_nbr']==32)][['city_state','store_nbr']]
city_state.value_counts()

The highest and lowest sales store numbers are present in city_state Quito_Pichincha and Guayaquil_Guayas respectively.

We will now study the average sales across different city_states.

In [ ]:
c=df.groupby(['city_state'])['sales'].mean().sort_values(ascending=False)
c=pd.DataFrame(c)
ax,f=plt.subplots(figsize=(30,20))
sns.barplot(x='sales',y=c.index,data=c,ax=None,ci=None)

From here as well, we can see that Quito_Pichincha has reported the highest sales.

In [ ]:
f,ax=plt.subplots(figsize=(20,10))
g=df.groupby(['type_locale'])['sales'].mean()

g.plot.pie(autopct='%1.1f%%',ax=None)

Additional_locale reports highest sales.

# **Predictive Analysis**

In [ ]:
#splitting the train dataset into train and test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns != 'sales'], train['sales'], test_size=0.33, random_state=42)

In [ ]:
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# one-hot encode the categorical features
cat_attribs = ['id','date','family']
full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)], remainder='passthrough')

encoder = full_pipeline.fit(X_train)
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)
test=encoder.transform(test)
# train the model
model = XGBRegressor(n_estimators=10, max_depth=20, verbosity=2)
model.fit(X_train, y_train)

# extract the training set predictions
model.predict(X_train)
# array([7.0887003, 3.7923286, 7.0887003], dtype=float32)

# extract the test set predictions
model.predict(X_test)
# array([7.0887003, 7.0887003], dtype=float32)

In [ ]:
model.score(X_test, y_test) #accuracy
#accuracy_score is for classification tasks only. For regression you should use something different
#Where X_test is sample, y_test is corresponding ground truth values. It will compute predictions inside
#R-squared can be computed by calling score function


In [ ]:
pred1=model.predict(test)
pred=pd.DataFrame(pred1)


pred.rename(columns={0:'sales'},inplace=True)
pred.index=test1['id']
pred

In [ ]:
pred.to_csv('submission.csv')